In [1]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools
import pygsheets as pygs
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd

In [2]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [3]:
g_drive_cred = r'C:\Users\darius kay\Dev\gitlab\test\python\credentials.json'
workbook = "https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414"

def clear_data_range(workbook_name, tab_name, data_range):
    start_end = str(data_range).split(':')
    start = start_end[0]
    print(start)
    end = start_end[1]
    print(end)
    print(g_drive_cred)
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)
    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)
    # select the first sheet
    wks = ws.worksheet_by_title(tab_name)
    print(ws.id)
    wks.clear(start=start, end=end)
    return

def pd_to_gsheets(workbook_name, sheet_name, df, row_num, col_num):
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)

    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)

    # select the first sheet
    wks = ws.worksheet_by_title(sheet_name)

    # update the designated sheet with df, starting at cell B2.
    wks.set_dataframe(df, (row_num, col_num))

    return

In [4]:
def date_checker(view, date, schedule):
    if view == 'forecast':
        if schedule == 'week':
            try:
                return date + timedelta(days=21)
            except TypeError:
                return
#         elif schedule == 'month':
#             try:
#                 return (date + timedelta(days=21)) + MonthEnd(0)
#             except TypeError:
#                 return
    else:
        return date

In [5]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [32]:
# Primary weekly forecast query.
weekly_forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC"

# Actual weekly primary query.
weekly_actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC"

In [33]:
# Monthly actual primary query.
monthly_actual_query = "SELECT fulfillment_month_ended month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_month_ended) >= 2020 ORDER BY 1 DESC"

# Primary monthly forecast query.
monthly_forecast_query = "SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', LAST_DAY(fulfillment_week_ended + 21)) >= 2020 ORDER BY 1 DESC"

In [34]:
# Weekly Forecasted qty returned/exchanged query.
weekly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Monthly Forecasted qty returned/exchanged query.
monthly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s AND plant IN %s THEN 1 ELSE 0 END)"

In [35]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product"

# Edit for cushion only non-cover.
product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product"

In [36]:
sizes = "CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '%6%x%9%' THEN '6x9' END size"

In [38]:
query_params = {
#     'send_back_product_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
#     'send_back_product_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
#                           'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                           'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                               'return': "('return')", 'exchange': "('exchange')"},
#                             'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
#     'send_back_product_size_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
#                                 'size': {'all': "'all' size", 'size_categories': sizes}},
    'send_back_product_size_monthly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}}
}

In [44]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
#     print(key)
    tab_df = pd.DataFrame()
    combinations = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    for combination in combinations:
        for combo in combination:
            keys = list(keys)
            grouping = ""
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                grouping = f"{grouping}, {i+2}"
#                 print(keys[i], grouping)
                if i == 0:
                    query = query_params[key][keys[i]][combo[keys[i]]][0]
#                     print(query)
                    calculation = query_params[key][keys[i]][combo[keys[i]]][1]
#                     print(calculation)
                    params = f"'{combo[keys[i]]}' AS {keys[i]}"
#                     print(params)
                else:
                    if keys[i] == 'send_back__type':
                        send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    elif keys[i] == 'plant':
                        calculation = calculation %(send_back_type, str(query_params[key][keys[i]][combo[keys[i]]]))
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
            query = query %(params, calculation, calculation, grouping)
            print(query)
#             print(calculation)
#             print(params)
#             print(grouping)
            df = pd.read_sql(query, conn)
            tab_df = tab_df.append(df)
    
    if 'weekly' in key:
        # Add 21 days to week_ended for forecast only.
        tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended'], 'week'), axis=1)

        # Concat first four columns.
        keys.insert(0, 'week_ended')
        uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
        tab_df.insert(loc=0, column='uid', value=uid)
    elif 'monthly' in key:
        # Add 21 days to month_ended for forecast only.
#         tab_df['month_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['month_ended'], 'month'), axis=1)
        if 'week_ended' in tab_df.columns:
            tab_df = tab_df.rename(columns={'week_ended': 'month_ended'})
        # Concat first four columns.
        keys.insert(0, 'month_ended')
        uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
        tab_df.insert(loc=0, column='uid', value=uid)

    clear_data_range(workbook, key, "A1:J100000")
    pd_to_gsheets(workbook, key, tab_df, 1, 1)

SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, 'all' size, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size I

SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'return' send_back__type, 'CHI' plant, 'all' size, SUM(CASE WHEN send_back__type in ('return') AND plant IN ('CHI') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return') AND plant IN ('CHI') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'return' send_back__type, 'CHI' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '%6%x%9%' THEN '6x9' END size, SUM(CASE WHEN send_back__type 

SELECT fulfillment_week_ended week_ended, 'actual' AS view, CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product, 'send_back_both' send_back__type, 'all' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '%6%x%9%' THEN '6x9' END size, SUM(CASE WHEN send_back__

SELECT fulfillment_week_ended week_ended, 'actual' AS view, CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product, 'return' send_back__type, 'all' plant, 'all' size, SUM(CASE WHEN send_back__type in ('return') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DE

SELECT fulfillment_week_ended week_ended, 'actual' AS view, CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product, 'exchange' send_back__type, 'all' plant, 'all' size, SUM(CASE WHEN send_back__type in ('exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER B

...

SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product, 'return' send_back__type, 'LA' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '%6%x%9%' THEN '6x9' END size, SUM(CASE WHEN send_back__type IN

SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product, 'exchange' send_back__type, 'LA' plant, 'all' size, SUM(CASE WHEN send_back__type IN ('exchange') AND plant IN ('LA') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type IN ('exchange') AND plant IN ('LA') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1,

SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product, 'send_back_both' send_back__type, 'all' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round' WHEN size ILIKE '%6%r%' THEN '6-Round' WHEN size ILIKE '%2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '%9%x%12%' THEN '9x12' WHEN size ILIKE '%8%x%10%' THEN '8x10' WHEN size ILIKE '%5%x%7%' THEN '5x7' WHEN size ILIKE '%2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '%6%x%9%' THEN '6x9' END size, SUM(CASE WHEN send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch

SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product, 'return' send_back__type, 'LA' plant, 'all' size, SUM(CASE WHEN send_back__type IN ('return') AND plant IN ('LA') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type IN ('return') AND plant IN ('LA') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC
SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product, 'return' send_back__type, 'LA' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' THEN '8-Round'

SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product, 'exchange' send_back__type, 'CHI' plant, 'all' size, SUM(CASE WHEN send_back__type IN ('exchange') AND plant IN ('CHI') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type IN ('exchange') AND plant IN ('CHI') AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= 2020 ORDER BY 1 DESC
SELECT fulfillment_week_ended week_ended, 'forecast' AS view, CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product, 'exchange' send_back__type, 'CHI' plant, CASE WHEN size ILIKE '%3%x%5%' THEN '3x5' WHEN size ILIKE '%8%r%' TH

In [ ]:
(tab_df['month_ended'][0] + timedelta(days=21)) + MonthEnd(0)

In [31]:


# clear_data_range(workbook, key, "A1:J100000")
# pd_to_gsheets(workbook, key, tab_df[pd.DatetimeIndex(tab_df['week_ended']).year >= 2020], 1, 1)

A1
J100000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I


In [40]:
keys

['month_ended', 'view', 'product', 'send_back__type', 'plant', 'size']

In [46]:
clear_data_range(workbook, key, "A1:J100000")
pd_to_gsheets(workbook, key, tab_df, 1, 1)

A1
J100000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I


In [45]:
tab_df

,uid,month_ended,view,product,send_back__type,plant,size,qty_returned,qty_sold,rate
0,2021-01-03_actual_all_send_back_both_all_all,2021-01-03,actual,all,send_back_both,all,all,4,13333,0.000300
1,2020-12-27_actual_all_send_back_both_all_all,2020-12-27,actual,all,send_back_both,all,all,291,33925,0.008578
2,2020-12-20_actual_all_send_back_both_all_all,2020-12-20,actual,all,send_back_both,all,all,1021,52216,0.019553
3,2020-12-13_actual_all_send_back_both_all_all,2020-12-13,actual,all,send_back_both,all,all,1895,57889,0.032735
4,2020-12-06_actual_all_send_back_both_all_all,2020-12-06,actual,all,send_back_both,all,all,2364,55766,0.042391
...,...,...,...,...,...,...,...,...,...,...
859,2020-01-05_forecast_nan_exchange_N/A_nan,2020-01-05,forecast,None,exchange,N/A,None,0,1,0.000000
860,2020-01-05_forecast_classic_exchange_N/A_2.5x10,2020-01-05,forecast,classic,exchange,N/A,2.5x10,0,1207,0.000000
861,2020-01-05_forecast_nan_exchange_N/A_8x10,2020-01-05,forecast,None,exchange,N/A,8x10,0,139,0.000000
862,2020-01-05_forecast_nan_exchange_N/A_3x5,2020-01-05,forecast,None,exchange,N/A,3x5,0,288,0.000000
